In [67]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import math
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats
#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing
%matplotlib inline

Manually stopping mux scans caused "phantom" TRs to be added that are total junk. This code reads from the design files to find where the experiment should end, and crops the functional data accordingly. In the future I will not use random ITIs.

In [62]:
#preliminary stuff
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'scripts/sub_cb_mappings.txt'
os.chdir(home_dir)
time_to_drop = 9.0
time_to_pad = 10.0 #how much padding to have after the last event

#get subject list
sub_list = []
sub_to_cb = {}
with open(subj_file) as f:
    for line in f:
        (key,val) = line.strip().split(' ') 
        sub_list.append('fd_' + key)
        sub_to_cb['fd_' + key] = int(val)

In [27]:
def find_spikes(d, spike_thresh):
    slice_mean = d.mean(axis=0).mean(axis=0)
    t_z = (slice_mean - np.atleast_2d(slice_mean.mean(axis=1)).T) / np.atleast_2d(slice_mean.std(axis=1)).T
    spikes = np.abs(t_z)>spike_thresh
    spike_inds = np.transpose(spikes.nonzero())
    # mask out the spikes and recompute z-scores using variance uncontaminated with spikes.
    # This will catch smaller spikes that may have been swamped by big ones.
    d.mask[:,:,spike_inds[:,0],spike_inds[:,1]] = True
    slice_mean2 = d.mean(axis=0).mean(axis=0)
    t_z = (slice_mean - np.atleast_2d(slice_mean.mean(axis=1)).T) / np.atleast_2d(slice_mean2.std(axis=1)).T
    spikes = np.logical_or(spikes, np.abs(t_z)>spike_thresh)
    spike_inds = np.transpose(spikes.nonzero())
    return((spike_inds, t_z))

In [28]:
#for loc experiment the experiment ends exactly at 312 seconds, so I can just lop at 208 TRs
def crop_roi(sub):
    new_size = 208
    scans = glob.glob('data/' + sub + '/func/loc/*fc.nii.gz')
    for s in scans:
        if sub == 'fd_108' and scan == 'run1': #bad quality for this subject at the end
            new_size = 196
            
        new_scan = s[:-7] + '_crop.nii.gz'
        if not os.path.exists(new_scan):
            cmd = 'fslroi ' + s + ' ' + new_scan + ' 0 ' + str(new_size)
            os.system(cmd)


In [29]:
pool = multiprocessing.Pool(processes = 5)
pool.map(crop_roi,sub_list)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [83]:
#sim and ser experiments
def crop_sim_ser(sub):
    cond = 'ser'
    design_dir = home_dir + 'data/' + sub + '/design/'
    design_file = design_dir + cond + '.csv'
    d = np.recfromcsv(design_file)
    for run in range(1,4):
        
        #find last event from the timing file
        last_event_time = 0
        for line in d:
            if line[0] == run:
                if line[2] > last_event_time:
                    last_event_time = line[2]
        last_event_time = last_event_time + time_to_drop

        #get scan duration
        scan = abspath('data/' + sub + '/func/' + cond + '/run_' + str(run) + '_fc.nii.gz')
        img = nib.load(scan)
        scan_time = img.shape[3]*1.5

        #get duration that scan should be
        scan_diff = scan_time - last_event_time
        if scan_diff < 0: #bad news
            print sub

        new_scan_time = last_event_time + min(scan_diff,time_to_pad) #pad scan
        new_scan_time = new_scan_time/1.5 #back into scans
        new_scan_time = int(math.ceil(new_scan_time))
        
        new_scan = scan[:-7] + '_crop.nii.gz'
        if not os.path.exists(new_scan):
            cmd = 'fslroi ' + scan + ' ' + new_scan + ' 0 ' + str(new_scan_time)
            os.system(cmd)          

In [84]:
pool = multiprocessing.Pool(processes = 7)
pool.map(crop_sim_ser,sub_list)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]